In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F

이 코드의 최종 목적은 이미지를 입력받아 10개 클래스 중 하나로 분류하는 CNN(합성곱 신경망)을 만드는 것

1) 입력:

1채널 흑백 이미지 (예: MNIST 손글씨, 28×28)
** 채널이란 이미지에서 색상이나 특징 맵을 구분하는 축. 흑백이미지는 각 필셀이 밝기하나만 가지니까 1채널임. 예를들어 rgb 컬러 이미지는 픽셀마다 빨강, 초록, 파랑 3개의 값이 있어서 3채널임

2) 처리 과정:

두 번의 합성곱(Convolution) → 두 번의 풀링(MaxPooling) → 완전연결층(Fully Connected) 3개

결국 완전 연결층이 분류기 역할이고 합성곱 레이어들은 이미지에서 중유한 패턴을 찾는 전처리기임. 

활성화 함수(ReLU)로 비선형성 추가

3) 출력:

크기 (N, 10)인 텐서 (N=배치 크기, 10=클래스 수)

각 값은 "이 이미지가 해당 클래스일 가능성"을 의미

4) 학습 시:

실제 라벨과 비교해 손실(Loss) 계산 → 역전파로 가중치 학습



필터(=커널) 란?
필터는 이미지에서 특징 패턴을 감지하는 정사각형의 작은 행렬. 
convolution layer 는 이 필터를 이미지 전체에 슬 . 라 . 이 . 딩 하면서 각 위치마다 하나의 숫자를 만듦.
필터 하나가 이미지 전체를 훑고 나면 하나의 출력 채널 (피쳐 맵) 이 만들어짐
각 필터는 다른 패턴 ( 뭐 수평선 수직선 점 곡선 등.. ) 에 반응함. 

In [90]:
class Net(nn.Module):
    def __init__(self): #레이어를 정의함. 
        super(Net, self).__init__()
        # kernel
        self.conv1=nn.Conv2d(1,6,5) 
        # 1 : input img channel =>  6: 6개의 필터 / 5: 5*5 convolution ( 필터의 크기 의미)
        self.conv2=nn.Conv2d(6,16,5)
        # 6 인풋 img 채널 => 16 개의 필터 /  5: 5*5 크기의 필터
        
        #fc nn.Linear(in_features, out_features) 선형 변환 
        # weight 는 (out_features, in_features)
        # bias 는 out_features
        # y=wx+b
        # fc 가 연속적으로 있는 이유?
        # 각 fc 는 입력 특징을 다른 차원으로 선형 변환하는데 그 사이사이 ReLU 처럼 비선형성을 넣어주면 모델이 더 복잡(비선형성) 학습 가능함. 
        #최종 fc3 의 출력 (10) 은 클래스별 점수 (logits) 이고 이를 소프트맥스하여 함께 사용. 
        self.fc1=nn.Linear(16*5*5,120) #입력 400 을  120 으로 줄임
        self.fc2=nn.Linear(120,84) 
        self.fc3=nn.Linear(84,10) 
        
        
    def forward(self, input):
        # convolution layer c1:   self.conv1=nn.Conv2d(1,6,5) 
        # it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        #배치 사이즈는 보통 데이터를 네트워크에 한꺼번에 넣는 이미지 개수를 의미함. 
        
        
        # 첫번째 합성곱 + relu 
        # 1 채널 이미지가 6개의 필터를 거쳐서 6개의 서로 다른 피쳐맵을 만들어서 출력 텐서 모양은 n, 6, 28, 28 이 됨
        c1=F.relu(self.conv1(input))
        
        #첫 번째 max pooling. 2*2 풀링. -> 크기 절반. 
        #(N, 6, 14, 14) 중요한 정보만 남기는 것. 
        s2=F.max_pool2d(c1,(2,2))
        
        #두번째 합성곱+ reLU
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        #14-5+ 2*0 +1  
        c3=F.relu(self.conv2(s2))
        
        #두번쨰 max pooling
        # (N, 16, 10, 10) Tensor -> (N, 16, 5,5)
        # 2랑 (2,2) 랑 같음 ㅋ
        s4=F.max_pool2d(c3,2)
        
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        #펼치기 : 4차원 텐서를 2차원으로 쭉 펴서 변환해줘야힘. 
        s4=torch.flatten(s4,1)
        # 첫번 째 차원은 그대로 두고 (N, 16,5, 5) => (N,400) 하는거임. 
        
        #완전 연결층
        #f5: (N,400) tensor input/ (N,120) Tensor output
        # 위 init에 fc1 만들어놓음. 
        # fc 층은 1차원 벡터 입력 필요. 
        f5=F.relu(self.fc1(s4)) #400->120
        
        f6=F.relu(self.fc2(f5)) #120-> 84
        
        output=self.fc3(f6) #84 -> 10
        # ex) 0~9 손글 씨 숫자 
        return output
        
net=Net()
print(net) 
         

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


합성곱 출력 크기 계산 공식 (기본):
출력크기 = ( 입력크기 - 필터크기 + 2* 패딩 ) / 스트라이드 + 1

스트라이드 : 필터 움직이는 간격 ( 1이면 모든 위치에서 연산함. )
패딩: 입력 주변에 0으로 둘러싼 픽셀 수 
=> 패딩을 하는 이유 : 이미지 가장자리 정보 손실을 막고, 출력 크기 조절하기 위하여 

cf) 합성곱의 기본문제: 가장자리 픽셀 정보가 적어짐. 
필터가 이미지 맨 가장자리 픽셀에 닿을 때 " 필터 일부는 이미지 밖을 벗어나기 떄문에 그 부분은 연산할 수 없음" 그래서 가장자리쪽은 내부 픽셀보다 적은 횟수로만 연산에 참여해 정보가 덜 반영.

fc layer란?
뉴런들이 앞층의 모든 뉴러노가 연결되어있는 층
입력의 각 요소가 모두 출력의 각 뉴런과 연결 되어이씀.

y=wx+b

입력 벡터에 가중치 행렬을 곱해서 편향 벡터 더함. 

왜필요하냐?

cnn 에서 합성곱 층은 특징 추출하는 역할을 하는데, 
완전 연결층은 이 추출된 특징을 분류, 예측 같은 최종 작업에 맞게 변환하는 역할을 함. 


net.parameters() 쓰면 그 모델이 학습가능한 텐서가 나옴. 

In [91]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


(배치 크기, 채널 수, 높이, 너비) 순서로 크기를 지정한 4차원 텐서 생성

torch.randn은 평균 0, 표준편차 1인 정규분포에서 무작위 값을 뽑아요.

여기서는:

1 → 배치 크기(batch size) = 한 번에 한 장의 이미지만 전달

1 → 채널 수 = 흑백 이미지(Gray scale)

32, 32 → 이미지 크기 32x32 픽셀

즉, 32x32 크기의 흑백 이미지를 1장 무작위로 만든 것이에요.

In [92]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)
#out은 네트워크가 입력 input에 대해 예측한 결과를 의미

tensor([[-0.1104,  0.0589,  0.0333, -0.0351, -0.1095,  0.0660,  0.0238,  0.1284,
          0.0514, -0.0174]], grad_fn=<AddmmBackward0>)


In [93]:
net.zero_grad()
out.backward(torch.randn(1,10))

Loss Function 
(output, target)을 input 을 받아서 목표와의 차이를 계산함. 

In [94]:
output=net(input)
#타겟은 정답 데이터
target=torch.randn(10)
target=target.view(1,-1)

#손실함수로 mse 사용.
criterion=nn.MSELoss()

loss=criterion(output,target)


print(loss)

tensor(0.7395, grad_fn=<MseLossBackward0>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

pytorch 의 autograd 연산그래프 탐색코드.
loss.grad_fn?

1) loss 는 텐서임.
2) loss는 계산이 어떻게 되었는지 기록하는 함수를 갖고있음.
3) grad_fn 은 이 텐서가 어떻게 만들어졌는지 /  계산 그래프의 마지막 함수를 나타내는 속성. 

In [95]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) #LINEAR
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #RELU


<MseLossBackward0 object at 0x000001CBDB34A9B0>=> loss 를 만든 연산함수이름  
<AddmmBackward0 object at 0x000001CBDDBDF3A0>=> loss 바로 이전 연산 LINEAR 층에서 하는 계싼연산의 backward 함수 객체.  
<AccumulateGrad object at 0x000001CBDB34A9B0>=> 그 이전 연산   

backprop

In [96]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()


print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0050, -0.0069,  0.0086,  0.0071, -0.0030,  0.0080])


위의 역전파에 따라 weights update. 아래는 simple 한 sgd.

In [97]:
# weight=weight-learning_rate*gradient
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

다양한 optimizer 를 쓰려면 package 써야됨.  
파라미터 업데이트. 
forward→backward→파라미터 업데이트 루프를 한 번씩 돌리는 게 정상


In [ ]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.01)

# in my training loop:
optimizer.zero_grad()
#역전파 전 기울기 초기화하는거임. 기본적으로 python 은 기존 grad 에 새로운 grad accumulate 함. 
# 즉 매번 .backward() 호출할 떄마다 누적됨. 
#so, 매 step 마다 초기화 해줘야 정확한 grad 업뎃 가능.


# !!!!!!!!!!1

#순전파 (예측값 계산)
output=net(input)
#손실 계산
loss=criterion(output,target)
#역전파
loss.backward()



# 파라미터 복사 (업데이트 전)
params_before = {}
for name, param in net.named_parameters():
    params_before[name] = param.data.clone()

# optimizer step (업데이트)!!!!!!!! 
optimizer.step()

# 파라미터 변화 출력z
for name, param in net.named_parameters():
    diff = param.data - params_before[name]
    print(f"{name} changed by:")
    print(diff)
  



conv1.weight changed by:
tensor([[[[-2.0929e-04,  4.8667e-05,  1.2733e-05, -2.1085e-05, -1.7181e-05],
          [ 2.0564e-06, -4.6909e-05, -2.9013e-05,  1.6853e-04,  7.4327e-05],
          [ 8.1271e-05, -1.4084e-05, -6.3166e-05,  1.0383e-04,  1.3530e-05],
          [-3.7104e-05,  1.0008e-05,  6.2510e-05, -5.7966e-06,  1.4083e-05],
          [ 1.3275e-04, -3.3885e-05, -1.2259e-04, -3.0078e-05,  7.9588e-05]]],


        [[[ 1.1796e-04, -3.9125e-06, -5.0887e-06, -8.6412e-05, -1.1364e-04],
          [ 3.3513e-05, -6.4969e-06, -3.9786e-06, -7.6085e-05,  1.0831e-04],
          [-6.3464e-05,  8.1435e-05, -4.1887e-05,  8.6337e-05,  4.8727e-06],
          [ 6.5789e-06, -7.5400e-06,  7.3150e-05, -1.2352e-04,  8.9869e-05],
          [-6.6608e-06, -1.2130e-05,  9.4198e-05,  3.0801e-05,  5.5164e-05]]],


        [[[-9.9458e-05, -3.2485e-05,  1.2353e-05, -1.9461e-05,  1.1861e-05],
          [ 4.3940e-05,  2.9460e-05,  2.9236e-05,  9.6902e-05, -4.2990e-06],
          [-2.9773e-05,  1.8205e-04, -2.083